In [1]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

In [2]:
from glob import glob

In [3]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go

In [38]:
files = glob(f'data/STK/5_mins/TSLA/*.csv')
data_list = []
for file in files:
    data_list.append(pd.read_csv(file))
df = pd.concat(data_list).reset_index(drop=True)

* check the trend change on before 12

In [39]:
df['date'] = pd.to_datetime(df['date'])
df.index = df['date']
df = df.between_time('21:30', '23:59')
df['time'] = df['date'].dt.time
df['date'] = df['date'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [40]:
valid_dates = df['date'].value_counts()

In [41]:
valid_dates = list(valid_dates[valid_dates > 3].index)

In [42]:
df = df[df['date'].isin(valid_dates)]

In [43]:
df.head()

,date,open,high,low,close,volume,barCount,average,time
date,,,,,,,,,
2021-04-30 21:30:00,2021-04-30,667.59,672.73,666.14,670.00,5348,2166,669.483,21:30:00
2021-04-30 21:35:00,2021-04-30,670.03,672.47,667.59,671.97,3712,1920,670.124,21:35:00
2021-04-30 21:40:00,2021-04-30,672.05,673.85,671.04,672.55,2235,1231,672.513,21:40:00
2021-04-30 21:45:00,2021-04-30,672.68,677.68,672.31,677.54,3257,1698,675.148,21:45:00
2021-04-30 21:50:00,2021-04-30,677.66,681.08,677.39,680.22,5305,2936,679.477,21:50:00


# Candle Chart

In [10]:
dates = sorted(df['date'].unique())

In [11]:
fig = go.Figure()

for date in dates:
    data = df[df['date'] == date]
    fig.add_trace(go.Candlestick(x=data['time'],
            open=data['open'],
            high=data['high'],
            low=data['low'],
            close=data['close'],
                                name = date))

import plotly.io as pio
pio.renderers.default = 'browser'
pio.show(fig)

# Line Chart

In [44]:
line_data = df[['date', 'time' ,'average', 'open']].copy()

In [45]:
line_data.reset_index(drop=True, inplace=True)

In [46]:
open_price = line_data.sort_values(['date', 'time']).groupby(['date'])['open'].first().reset_index()

In [47]:
# Find the min open price
# min_open_price = line_data['open'].min()

In [48]:
line_data = line_data.merge(open_price, on=['date'], suffixes=('_actual', '_day'))

In [49]:
# To bring all open price to 0, subtract open price from average
line_data['avg_nor'] = line_data['average'] - line_data['open_day']

In [50]:
dates = sorted(line_data['date'].unique())

In [51]:
fig = go.Figure()

for date in dates:
    data = line_data[line_data['date'] == date]
    fig.add_trace(go.Scatter(x=data['time'], y=data['avg_nor'], name = date))

import plotly.io as pio
pio.renderers.default = 'browser'
pio.show(fig)